# Imports
The following imports are needed extract and transform data to plot metrics and make analysis.

* pandas: used for creating dataframes
* numpy: used for doing aritmetic calculations like averages
* matplotlib.pyplot: used for creating plots
* re: pyhtong library for regex, used for finding matches in strings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re # python regex library
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

# Upload File

In [4]:
from google.colab import files

csv = files.upload()

Saving Datos.csv to Datos.csv


# Load CSV

In [ ]:
dataframe = pd.read_csv("Datos.csv")
len(dataframe) # print number of rows

9595

# Transformations
I will try to discard the least amount of rows with the following transformations

## Empty Ratings
They will be assigned 0, in some analysis they will be discarded but keeping them lets us include the rest of the information in other columns of the rows with not rating

In [ ]:
dataframe["Rating"] = pd.to_numeric(dataframe["Rating"].fillna(0.0))

,Course Title,Rating,Level,Duration,Schedule,Review,What you will learn,Skill gain,Modules,Instructor,Offered By,Keyword,Course Url
9590,Power Onboarding,4.1,Beginner level,Approx. 7 hours to complete,Flexible schedule,36 reviews,NaN,[],"['Week 1 - Get Ready', 'Session 2 - Get Set', ...",['William J. White'],['Northwestern University'],Social Sciences,https://www.coursera.org/learn/power-onboarding
9591,Implementing Parent Feedback with Google Forms,4.9,Beginner level,NaN,NaN,77 reviews,Creating engaging parent surveys using Google ...,"['Education', 'online tools', 'engagement', 'e...",['Learn step-by-step'],['Rachel Lindsay'],['Coursera Project Network'],Social Sciences,https://www.coursera.org/projects/education-fe...
9592,"""Making"" Progress Teach-Out",4.2,Beginner level,Approx. 5 hours to complete,Flexible schedule,10 reviews,NaN,[],"['Week 1', 'WEEK 2']",['Pablo Palomino'],['Emory University'],Social Sciences,https://www.coursera.org/learn/makingprogress
9593,Use Microsoft PowerPoint to Create a Visual Sc...,4.5,Beginner level,1 hour,NaN,11 reviews,Learn about choice boards and their use within...,"['Teaching tools', 'elementary teaching', 'eng...",['Learn step-by-step'],['Rachel Lindsay'],['Coursera Project Network'],Social Sciences,https://www.coursera.org/projects/use-microsof...
9594,Créer du contenu pour les réseaux sociaux avec...,0.0,Beginner level,NaN,NaN,NaN,Vous découvrirez les fonctionnalités de Prezi ...,"['Social Media Marketing', 'Social Media', 'Co...",['Learn step-by-step'],['Sara Nefzaoui'],['Coursera Project Network'],Social Sciences,https://www.coursera.org/projects/creer-conten...


## Duration
I use regex to find time unit matches. Priority to find string matches, from more priority to less priority, will be month, week, hour and minutes, minutes.

* We will transform everything to hours. Months has an average of 4.33 weeks, we will use this info to calcualte the total hours. Minutes will be given in their hour fraction, for example 15 minutes is 0.25 hours
* For debugging purposes I will return a tupple which will have the total ammount of hours and the greatest unit of time we used to calculate hours
* Empty strings will be [0.0 "hour"]
* Strings that don't match our regular expressions will be declared as [0.0, "noFormat"]

In [ ]:
def parse_duration(text):
    if not isinstance(text, str) or not text.strip():
        return (0.0, "hour")

    text = text.lower()

    # Check for months first
    month_match = re.search(r'(\d+)\s*month', text)
    months = None
    if month_match:
        months = float(month_match.group(1))
    
    # Check for hours and minutes
    hour_match = re.search(r'(\d+)\s*hour', text)
    hours = None
    if hour_match:
        hours = int(hour_match.group(1))

    if months and hours:
        total_weeks = months * 4.33 # months have an average of 4.33 weeks
        total_hours = total_weeks * hours # courses that last months will how detail many hours a week are expected to study
        return(total_hours, "month")

    week_match = re.search(r'(\d+)\s*week', text)
    weeks = None
    if week_match:
        weeks = float(week_match.group(1))
        
        if hours:
            total_days = weeks * 7
            total_hours = total_days * hours # courses that last weeks will detail how many hours a week is needed
            return(total_hours, "week")

    minute_match = re.search(r'(\d+)\s*minute', text)
    minutes = None

    if minute_match:
        minutes = int(minute_match.group(1))

    # case when there is no hours just minuts
    if not hours and isinstance(minutes, int) and minutes > 0:
        hour_fraction = minutes / 60
        return (hour_fraction, 'minute')

    # Apply rounding rules based on minutes
    # if minutes == 0:
    #     amount = float(hours)
    # elif minutes <= 30:
    #     amount = float(hours) + 0.5
    # else:
    #     amount = float(hours + 1)

    # If no valid time info found
    if not hours and not minutes:
        return (0.0, "noFormat")

    hour_fraction = minutes / 60 if minutes else 0.0
    hours = hours + hour_fraction
    return (hours, 'hour')

dataframe[['ammountOfHours', 'courseLengthBaseUnit']] = dataframe['Duration'].apply(lambda x: pd.Series(parse_duration(x)))

dataframe.loc[(dataframe['ammountOfHours'] == 0.0) & (dataframe['courseLengthBaseUnit'] == "hour")].head()

,Course Title,Rating,Level,Duration,Schedule,Review,What you will learn,Skill gain,Modules,Instructor,Offered By,Keyword,Course Url,ammountOfTime,unit
16,Create your UX portfolio with Adobe Portfolio,0.0,Beginner level,NaN,NaN,NaN,In this project you will learn how to create a...,"['Share Content', 'Search Engine Optimization ...",['Learn step-by-step'],['Nicole Alvelo'],['Coursera Project Network'],Arts and Humanities,https://www.coursera.org/projects/create-your-...,0.0,hour
235,Adobe Photoshop Para Principiantes: Efectos y ...,0.0,Beginner level,NaN,NaN,NaN,Aplica filtros inteligentes a una imagen.Incor...,"['Adobe Photoshop', 'Photography', 'Photo Enha...",['Learn step-by-step'],['Kevin Rodriguez'],['Coursera Project Network'],Arts and Humanities,https://www.coursera.org/projects/adobe-photos...,0.0,hour
398,Cómo crear historias para Instagram y Facebook...,4.2,Beginner level,NaN,NaN,18 reviews,En este proyecto aprenderás a utilizar Canva p...,"['Social Media Marketing', 'Social Media', 'Co...",['Learn step-by-step'],['Monica I. Heredia Rodriguez'],['Coursera Project Network'],Arts and Humanities,https://www.coursera.org/projects/como-crear-h...,0.0,hour
447,Edita tus fotos para las redes sociales con Easil,0.0,Beginner level,NaN,NaN,NaN,Crea un proyecto fotográfico individual en Eas...,"['Social Media Marketing', 'Photography', 'Soc...",['Learn step-by-step'],['Michelle Flores'],['Coursera Project Network'],Arts and Humanities,https://www.coursera.org/projects/edita-fotos-...,0.0,hour
448,Bearbeite deine Fotos für soziale Medien mit E...,0.0,Beginner level,NaN,NaN,NaN,Designe dein individuelles Fotoprojekt für soz...,"['Social Media Marketing', 'Photography', 'Gra...",['Learn step-by-step'],['Michelle Flores'],['Coursera Project Network'],Arts and Humanities,https://www.coursera.org/projects/bearbeite-fo...,0.0,hour


3. "Skill gain" may be used to determine most attractive skills for customers, so I will separate them
* If list is empty then default value will be "not_defined"

In [8]:
import ast

dataframe['Skill gain'] = dataframe['Skill gain'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) # turn it into a list
dataframe['Skill gain'] = dataframe['Skill gain'].apply(lambda x: x if x else ['NA']) # replace empty lists

dataframe = dataframe.explode('Skill gain').rename(columns={'Skill gain': 'skillLabel'}) # break list down into individual rows

dataframe.loc[dataframe["skillLabel"] == "NA"].head()

,Course Title,Rating,Level,Duration,Schedule,Review,What you will learn,skillLabel,Modules,Instructor,Offered By,Keyword,Course Url,ammountOfTime,unit
1,Modern American Poetry,4.4,Beginner level,Approx. 34 hours to complete,Flexible schedule,100 reviews,NaN,NA,"['Orientation', 'Module 1', 'Module 2', 'Modul...",['Cary Nelson'],['University of Illinois at Urbana-Champaign'],Arts and Humanities,https://www.coursera.org/learn/modern-american...,34.0,hour
2,Pixel Art for Video Games,4.5,Beginner level,9 hours (approximately),Flexible schedule,227 reviews,NaN,NA,"['Week 1: Introduction to Pixel Art', 'Week 2:...","['Andrew Dennis', 'Ricardo Guimaraes']",['Michigan State University'],Arts and Humanities,https://www.coursera.org/learn/pixel-art-video...,9.0,hour
3,Distribución digital de la música independiente,0.0,Beginner level,Approx. 8 hours to complete,Flexible schedule,NaN,NaN,NA,"['Semana 1', 'Semana 2', 'Semana 3', 'Semana 4']",['Eduardo de la Vara Brown.'],['SAE Institute México'],Arts and Humanities,https://www.coursera.org/learn/distribucion-di...,8.0,hour
5,So You Think You Know Tango?,4.6,Beginner level,Approx. 5 hours to complete,Flexible schedule,107 reviews,Gain an appreciation for the Argentine Tango a...,NA,['Module 1: The Many Dimensions of Tango and T...,['Kristin Wendland'],['Emory University'],Arts and Humanities,https://www.coursera.org/learn/tango,5.0,hour
6,The Politics of Skepticism,4.5,Intermediate level,Approx. 13 hours to complete,Flexible schedule,38 reviews,NaN,NA,"['Political Origins', 'Skepticism and Religion...","['Tim De Mey', 'Wiep van Bunge']",['Erasmus University Rotterdam'],Arts and Humanities,https://www.coursera.org/learn/erasmus-philoso...,13.0,hour


## Level
* I will remove the redundant word "level" for readability purposes
* The string will be lowercase
* Empty values will be replaced with "not_defined"

In [ ]:
dataframe['Level'] = (
    dataframe['Level']
    .fillna('')                             # Handle None values
    .str.lower()                            # Convert to lowercase
    .str.replace('level', '', regex=False) # Remove the word 'level'
    .str.strip()                            # Remove leading/trailing spaces
    .replace('', 'not_defined')            # Replace empty strings with default
)

dataframe.loc[pd.isna(dataframe["Level"])].head() # should print no values

,Course Title,Rating,Level,Duration,Schedule,Review,What you will learn,skillLabel,Modules,Instructor,Offered By,Keyword,Course Url,ammountOfTime,unit


## Reviews
* Will be transformed to a numeric value using regex

In [ ]:
dataframe["Review"] = (
    dataframe["Review"]
    .fillna('0')
    .str.extract(r'([\d,]+)')[0]                     # Extract digits with commas
    .str.replace(',', '', regex=False)
    .fillna('0')
    .astype(int)
)

dataframe.loc[pd.isna(dataframe["Review"])] # should print no values

,Course Title,Rating,Level,Duration,Schedule,Review,What you will learn,skillLabel,Modules,Instructor,Offered By,Keyword,Course Url,ammountOfTime,unit


## Schedule
* Column will be Renamed to "SchedFlexibility" for comprehensibility purposes
* Value will be "true" or "false" boolean values

In [ ]:
dataframe["Schedule"] = dataframe['Schedule'].apply(
    lambda x: isinstance(x, str) and 'flexible' in x.lower()
)

dataframe.rename(columns={'Schedule': 'SchedFlexibility'}, inplace = True)

,Course Title,Rating,Level,Duration,SchedFlexibility,Review,What you will learn,skillLabel,Modules,Instructor,Offered By,Keyword,Course Url,ammountOfTime,unit
0,Fashion as Design,4.8,beginner,20 hours (approximately),True,2813,NaN,Art History,"['Introduction', 'Heroes', 'Silhouettes', 'Cou...","['Anna Burckhardt', 'Paola Antonelli', 'Michel...",['The Museum of Modern Art'],Arts and Humanities,https://www.coursera.org/learn/fashion-design,20.0,hour
0,Fashion as Design,4.8,beginner,20 hours (approximately),True,2813,NaN,Art,"['Introduction', 'Heroes', 'Silhouettes', 'Cou...","['Anna Burckhardt', 'Paola Antonelli', 'Michel...",['The Museum of Modern Art'],Arts and Humanities,https://www.coursera.org/learn/fashion-design,20.0,hour
0,Fashion as Design,4.8,beginner,20 hours (approximately),True,2813,NaN,History,"['Introduction', 'Heroes', 'Silhouettes', 'Cou...","['Anna Burckhardt', 'Paola Antonelli', 'Michel...",['The Museum of Modern Art'],Arts and Humanities,https://www.coursera.org/learn/fashion-design,20.0,hour
0,Fashion as Design,4.8,beginner,20 hours (approximately),True,2813,NaN,Creativity,"['Introduction', 'Heroes', 'Silhouettes', 'Cou...","['Anna Burckhardt', 'Paola Antonelli', 'Michel...",['The Museum of Modern Art'],Arts and Humanities,https://www.coursera.org/learn/fashion-design,20.0,hour
1,Modern American Poetry,4.4,beginner,Approx. 34 hours to complete,True,100,NaN,NA,"['Orientation', 'Module 1', 'Module 2', 'Modul...",['Cary Nelson'],['University of Illinois at Urbana-Champaign'],Arts and Humanities,https://www.coursera.org/learn/modern-american...,34.0,hour


## Cleaing
Removing duplicates is simple since we should not have any NaN(numpy) values any more

In [ ]:
unique_entries = dataframe.drop_duplicates()

# Vizualizations
## Bar Plot
Quantity of courses by "Level" bar plot. We can appreciatte that there is a good ammount of courses without a designated level

In [ ]:
# count how many unique course titles exist per level
course_count_by_level = unique_entries.groupby('Level').size().reset_index(name='Course Count')

course_count_by_level = course_count_by_level.sort_values('Course Count', ascending=True)

plt.figure(figsize=(8, 5))
plt.bar(course_count_by_level['Level'], course_count_by_level['Course Count'], color='skyblue')
plt.xlabel('Levels')
plt.title('Number of Courses by Level')
plt.tight_layout()
plt.show()


## Horizontal Bar Plot
Quantity of courses by "Keyword"(category) bar plot

In [ ]:
# Count how many unique courses in each categories exist
course_count_by_category = unique_entries.groupby('Keyword').size().reset_index(name='Course Count')

course_count_by_category = course_count_by_category.sort_values('Course Count', ascending=True)

plt.figure(figsize=(24, 8))
plt.barh(course_count_by_category['Keyword'], course_count_by_category['Course Count'], color='skyblue')
plt.xlabel('Course Quantity')
plt.title('Number of Courses by Category')
plt.tight_layout()
plt.show()

## Scatter Plot(dispercion)
Shows the relationship between the course duration and the number of reviews

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(x= unique_entries['ammountOfHours'], y= unique_entries['Review'], alpha=0.5, color='purple')
plt.xlabel('Course Duration (hours)')
plt.ylabel('Number of Reviews')
plt.title('Course Duration vs Review Count')
plt.tight_layout()
plt.show()

## Histogram Plot(Histograma)
Shows distribution of the courses duration(time ammount in hours), this shows that there is more courses that last a few hours, there is more than 4,000 courses that last maybe less than 5 or 8 hours 

In [ ]:
data = unique_entries['ammountOfHours']

plt.hist(data, bins=30, color='skyblue', edgecolor='black')

plt.xlabel('Hours')
plt.ylabel('Number of Courses')
plt.title('Courses by Hours')

# Display the plot
plt.show()

In [ ]:
dataframe.loc[dataframe['ammountOfHours'] > 14]

## Boxplot Plot
Plot by "Level"

In [ ]:
# drop columns with Ratings with values of 0.0, which is what I assigned to columns 
# that didn't have rating, because it could mess up this analysis
unique_courses = unique_entries[unique_entries['Rating'] != 0.0]

plt.figure(figsize=(12, 8))
unique_courses.boxplot(column="Rating", by="Level", notch=True)
plt.title('Course Ratings by Level')
plt.suptitle('')  # Remove the automatic suptitle from pandas boxplot
plt.xlabel('Level')
plt.ylabel('Rating')
plt.tight_layout()
plt.show()


In [ ]:
3. "Skill gain" may be used to determine most attractive skills for customers, so I will separate them
* If list is empty then default value will be "not_defined"



import ast

dataframe['Skill gain'] = dataframe['Skill gain'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) # turn it into a list
dataframe['Skill gain'] = dataframe['Skill gain'].apply(lambda x: x if x else ['NA']) # replace empty lists

dataframe = dataframe.explode('Skill gain').rename(columns={'Skill gain': 'skillLabel'}) # break list down into individual rows

dataframe.loc[dataframe["skillLabel"] == "NA"].head()

# Analysis
I will use Numpy to be a able to perform some operations to get the values I want to find

In [ ]:
# unique_courses = dataframe.groupby(['Level', 'Course Title','Instructor', 'Review', 'ammountOfHours']).size().reset_index(name="count")
# unique_courses = dataframe.drop_duplicates(subset=['Level', 'Course Title', 'Instructor', 'Rating', 'Review', 'ammountOfHours'])

# # unique_courses removes all courses that have the, Level
# review_variations = unique_courses.groupby(['Course Title', 'Instructor', 'ammountOfHours']).size().reset_index(name='count')

# review_variations.loc[review_variations['count'] > 1]

# len(review_variations)

# average_ratings = unique_courses.groupby(['Level', 'Course Title', 'Instructor', 'ammountOfHours'])['Rating'].mean().reset_index(name='Average Rating')

# agg_courses = dataframe.groupby(['Level', 'Course Title', 'Instructor', 'ammountOfHours']).agg({
#     'Rating': 'mean',
#     'Review': 'sum'  # or 'mean' if you'd rather average review counts too
# }).reset_index()

# len(average_ratings)


# unique_courses.loc[unique_courses['count'] == 9]

# unique_courses = dataframe.drop_duplicates(subset=['Level', 'Course Title', 'Instructor', 'Rating', 'Review', 'ammountOfHours'])



# unique_entries = dataframe.drop_duplicates()

# Rows with Rating 0 are the ones I transformed but in this analysis they might
# affect the result so I will remove them
filtered_entries = unique_entries[unique_entries['Rating'] != 0]

# Group same courses and get the average rating
average_ratings = filtered_entries.groupby(['Level', 'Course Title', 'Instructor', 'ammountOfHours'])['Rating'].mean().reset_index(name='average')

# group = average_ratings.loc[average_ratings['average'] < 2]

print('Muestra de los promedios de los ratings por curso:')
print('')
display(average_ratings.head())


In [ ]:
unique_courses = dataframe.groupby(['Level', 'Course Title','Instructor', 'Review', 'ammountOfHours']).size().reset_index()
# # Now count how many unique course titles exist per level
# course_count_by_level = unique_courses.groupby('Level').size().reset_index(name='Course Count')

# course_count_by_level = course_count_by_level.sort_values('Course Count', ascending=True)
# unique_courses.loc[unique_courses[0] > 2]

# len(unique_courses)
# 6398

len(unique_courses)
# 6408

review_variations = unique_courses.groupby(['Course Title', 'Instructor', 'ammountOfHours'])['Review'].nunique().reset_index()

# # Step 2: Keep only course titles with more than 1 unique review value
review_variations = review_variations[review_variations['Review'] > 1]

display(review_variations)


## Highest and Lowest Ratings
We get the course with the highest rating average and the first course found that has the highest Rating entry and also do the same anallysis but to find the lowest rating average and rating entry

In [ ]:
greatest_average = average_ratings['average'].max()
greatest_average_course = average_ratings.loc[average_ratings['average'].idxmax()]
print('')
print('')
print('El promedio mas alto es de:', greatest_average,', el primer curso encontrado con este promedio se llama:', greatest_average_course['Course Title'])

lowest_average = average_ratings['average'].min()
lowest_average_course = average_ratings.loc[average_ratings['average'].idxmin()]
print('El promedio mas bajo es de:', lowest_average,', el primer curso encontrado con este promedio se llama:', lowest_average_course['Course Title'])

greatest_rating = unique_entries['Rating'].max()
greatest_rating_course = unique_entries.loc[unique_entries['Rating'].idxmax()]
print('El Rating mas alto es de:', greatest_rating,', el primer curso encontrado con este Rating se llama:', greatest_rating_course['Course Title'])

lowest_rating = unique_entries['Rating'].min()
lowest_rating_course = unique_entries.loc[unique_entries['Rating'].idxmin()]
print('El Rating mas bajo es de:', lowest_rating,', el primer curso encontrado con este Rating se llama:', lowest_rating_course['Course Title'])


# A nice thing to do would be analize how many rows are we losing because the
# rating was missing, this may help us indicate a customer that there is a very 
# bad issue in some place of our system, either the backed or frontend